In [1]:
!which python

/mnt/chicm/anaconda3/bin/python


In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle

In [2]:
'''
!pip install opencv-python
!pip install fastparquet
!pip install pyarrow
!pip install snappy
!conda install python-snappy -y
'''

'\n!pip install opencv-python\n!pip install fastparquet\n!pip install pyarrow\n!pip install snappy\n!conda install python-snappy -y\n'

In [3]:
!ls /mnt/chicm/data/bengali

bengaliai-cv19.zip	   test_image_data_3.parquet
class_map.csv		   train.csv
sample_submission.csv	   train_image_data_0.parquet
test.csv		   train_image_data_1.parquet
test_image_data_0.parquet  train_image_data_2.parquet
test_image_data_1.parquet  train_image_data_3.parquet
test_image_data_2.parquet


In [4]:
DATA_DIR = '/mnt/chicm/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [7]:
class_map_df.head()

,component_type,label,component
0,grapheme_root,0,ং
1,grapheme_root,1,ঃ
2,grapheme_root,2,অ
3,grapheme_root,3,আ
4,grapheme_root,4,ই


In [8]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [9]:
train_df.shape

(200840, 5)

In [10]:
len(train_df.image_id.unique())

200840

In [11]:
train_df.grapheme_root.value_counts()

72     5736
64     5596
13     5420
107    5321
23     5149
       ... 
130     144
158     143
102     141
33      136
73      130
Name: grapheme_root, Length: 168, dtype: int64

In [12]:
df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet')
df.head()

,image_id,0,1,2,3,4,5,6,7,8,...,32322,32323,32324,32325,32326,32327,32328,32329,32330,32331
0,Train_0,254,253,252,253,251,252,253,251,251,...,253,253,253,253,253,253,253,253,253,251
1,Train_1,251,244,238,245,248,246,246,247,251,...,255,255,255,255,255,255,255,255,255,254
2,Train_2,251,250,249,250,249,245,247,252,252,...,254,253,252,252,253,253,253,253,251,249
3,Train_3,247,247,249,253,253,252,251,251,250,...,254,254,254,254,254,253,253,252,251,252
4,Train_4,249,248,246,246,248,244,242,242,229,...,255,255,255,255,255,255,255,255,255,255


In [13]:
df.shape

(50210, 32333)

In [14]:
df2 = df.set_index('image_id')

In [15]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,32322,32323,32324,32325,32326,32327,32328,32329,32330,32331
image_id,,,,,,,,,,,,,,,,,,,,,
Train_0,254,253,252,253,251,252,253,251,251,253,...,253,253,253,253,253,253,253,253,253,251
Train_1,251,244,238,245,248,246,246,247,251,252,...,255,255,255,255,255,255,255,255,255,254
Train_2,251,250,249,250,249,245,247,252,252,252,...,254,253,252,252,253,253,253,253,251,249
Train_3,247,247,249,253,253,252,251,251,250,250,...,254,254,254,254,254,253,253,252,251,252
Train_4,249,248,246,246,248,244,242,242,229,225,...,255,255,255,255,255,255,255,255,255,255


In [50]:
df2.index

Index(['Train_0', 'Train_1', 'Train_2', 'Train_3', 'Train_4', 'Train_5',
       'Train_6', 'Train_7', 'Train_8', 'Train_9',
       ...
       'Train_50200', 'Train_50201', 'Train_50202', 'Train_50203',
       'Train_50204', 'Train_50205', 'Train_50206', 'Train_50207',
       'Train_50208', 'Train_50209'],
      dtype='object', name='image_id', length=50210)

In [6]:
HEIGHT = 137
WIDTH = 236

In [7]:
#img = 255 - df.iloc[10, 1:].values.reshape(HEIGHT, WIDTH).astype(np.uint8)
#plt.imshow(img)

In [8]:
#img2 = cv2.resize(img, (256, 128))
#plt.imshow(img2)

In [9]:
import albumentations as albu

def get_train_augs(p=1.):
    return albu.Compose([
        #albu.HorizontalFlip(.5),
        albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5 ),
        albu.Blur(blur_limit=3, p=0.3),
        albu.OpticalDistortion(p=0.3),
        albu.GaussNoise(p=0.3)
        #albu.GridDistortion(p=.33),
        #albu.HueSaturationValue(p=.33) # not for grey scale
    ], p=p)

In [28]:
augs = get_train_augs(1.0)

In [31]:
img.shape

torch.Size([4, 1, 137, 236])

In [75]:
#x=augs(image=img)['image']

In [8]:
#plt.imshow(x)

In [10]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

HEIGHT = 137
WIDTH = 236

class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        #print(img.shape)
        if self.train_mode:
            augs = get_train_augs()
            img = augs(image=img)['image']
        
        img = np.expand_dims(img, axis=-1)
        
        img = transforms.functional.to_tensor(img)
        #img = transforms.functional.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
        if self.test_mode:
            return img
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic])

    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df = shuffle(train_df, random_state=1234)
    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    split_index = int(len(train_df) * 0.9)
    
    train = train_df.iloc[:split_index]
    val = train_df.iloc[split_index:]
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [12]:
#train_loader, val_loader = get_train_val_loaders(dev_mode=True)

(200840, 5)
(50210, 32332)


In [21]:
'''
for img, y in train_loader:
    print(img.size(), y.size())
    print(y)
    #print(img)
    #plt.imshow(img.squeeze()[0].numpy())
    break
'''

'\nfor img, y in train_loader:\n    print(img.size(), y.size())\n    print(y)\n    #print(img)\n    #plt.imshow(img.squeeze()[0].numpy())\n    break\n'

# model

In [8]:
#import pretrainedmodels

In [10]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [81]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [82]:
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=False).cuda()


In [83]:
#model.features(torch.randn((2, 3, 137, 236)).cuda()).size()

torch.Size([2, 2048, 5, 8])

In [84]:
#model.last_linear.in_features

2048

In [11]:
class BengaliNet(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        self.backbone = pretrainedmodels.__dict__[backbone_name](num_classes=1000, pretrained='imagenet')
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

    def logits(self, x):
        x = self.avg_pool(x)
        x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        x = self.conv0(x)
        #print(x.size())
        x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [12]:
MODEL_DIR = './models'
def create_model(args):
    model = BengaliNet(backbone_name=args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if args.predict and (not os.path.exists(model_file)):
        raise AttributeError('model file does not exist: {}'.format(model_file))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [23]:
'''
args = Namespace()
args.backbone = 'se_resnext50_32x4d'
args.ckp_name = 'best_model.pth'
args.predict = False

bnet = create_model(args)[0].cuda()
'''

model file: ./models/se_resnext50_32x4d/best_model.pth, exist: False


In [103]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [108]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

torch.Size([2, 3, 137, 236])


torch.Size([2, 186])

# train

In [40]:
round(1/9, 6)

0.111111

In [13]:
import numpy as np
import sklearn.metrics
import torch


def macro_recall(pred_y, y, n_grapheme=168, n_vowel=11, n_consonant=7):
    pred_y = torch.split(pred_y, [n_grapheme, n_vowel, n_consonant], dim=1)
    pred_labels = [torch.argmax(py, dim=1).cpu().numpy() for py in pred_y]

    #y = y.cpu().numpy()
    # pred_y = [p.cpu().numpy() for p in pred_y]

    recall_grapheme = sklearn.metrics.recall_score(pred_labels[0], y_grapheme, average='macro')
    recall_vowel = sklearn.metrics.recall_score(pred_labels[1], y_vowel, average='macro')
    recall_consonant = sklearn.metrics.recall_score(pred_labels[2], y_consonant, average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_score = np.average(scores, weights=[2, 1, 1])
    # print(f'recall: grapheme {recall_grapheme}, vowel {recall_vowel}, consonant {recall_consonant}, '
    #       f'total {final_score}, y {y.shape}')
    return final_score

def calc_metrics(preds0, preds1, preds2, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    
    
    return metrics

In [14]:
def accuracy(output, label, topk=(1,)):
    maxk = max(topk)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(label.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).sum().item()
        res.append(correct_k)
    return res

In [15]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    
    return loss0 + loss1 + loss2 #, loss0.item(), loss1.item(), loss2.item()

In [16]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2 = 0., 0., 0.
    preds0, preds1,preds2 = [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs = model(x)
            outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    
    return metrics
            

In [18]:
#train_loader, val_loader = get_train_val_loaders(batch_size=32, val_batch_size=128, dev_mode=True)

(200840, 5)
(50210, 32332)


In [87]:
type(model)

torch.nn.parallel.data_parallel.DataParallel

In [86]:
validate(model, val_loader)

RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 547, in __call__
    result = self.forward(*input, **kwargs)
  File "/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/data_parallel.py", line 146, in forward
    "them on device: {}".format(self.src_device_obj, t.device))
RuntimeError: module must have its parameters and buffers on device cuda:0 (device_ids[0]) but found one of them on device: cuda:1


(200840, 5)
(200840, 32332)
(180756, 5) (20084, 5)


model file: ./models/se_resnext50_32x4d/best_model.pth, exist: False


In [17]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [18]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [19]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [ ]:
#np.random.beta(1,1)

In [20]:
def train(args):
    global model

    if args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.0001)
    else:
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.0001)

    if args.lrs == 'plateau':
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=args.factor, patience=args.patience, min_lr=args.min_lr)
    else:
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.t_max, eta_min=args.min_lr)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    best_metrics = 0.
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)
    best_metrics = val_metrics[best_key]
    
    model.train()
    #optimizer.zero_grad()

    if args.lrs == 'plateau':
        lr_scheduler.step(best_metrics)
    else:
        lr_scheduler.step()
    train_iter = 0

    for epoch in range(args.num_epochs):
        train_loss = 0

        current_lr = get_lrs(optimizer)
        bg = time.time()
        for batch_idx, (img, targets) in enumerate(train_loader):
            train_iter += 1
            img, targets  = img.cuda(), targets.cuda()
            #img, targets = mixup(img, targets)
            batch_size = img.size(0)
          
            outputs = model(img)
            #loss = mixup_criterion(outputs, targets)
            loss = criterion(outputs, targets)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            #with amp.scale_loss(loss*batch_size, optimizer) as scaled_loss:
            #    scaled_loss.backward()
            
            #if batch_idx % 4 == 0:
            #    optimizer.step()
            #    optimizer.zero_grad()

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

            if train_iter > 0 and train_iter % args.iter_val == 0:
                #outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
                #preds0 = (torch.max(outputs[0], dim=1)[1]).cpu().numpy()
                #preds1 = (torch.max(outputs[1], dim=1)[1]).cpu().numpy()
                #preds2 = (torch.max(outputs[2], dim=1)[1]).cpu().numpy()
                #train_metrics = calc_metrics(preds0, preds1, preds2, targets.cpu().numpy())
                #print('train:', train_metrics)
                #save_model(model, model_file+'_latest')
                val_metrics = validate(model, val_loader)
                print('\nval:', val_metrics)
                
                if val_metrics[best_key] > best_metrics:
                    best_metrics = val_metrics[best_key]
                    save_model(model, model_file)
                    print('** saved')
                
                model.train()
                
                if args.lrs == 'plateau':
                    lr_scheduler.step(best_metrics)
                else:
                    lr_scheduler.step()
                current_lr = get_lrs(optimizer)
        
    

In [21]:
args = Namespace()
args.backbone = 'se_resnext50_32x4d'
args.ckp_name = 'best_model.pth'
args.predict = False
args.optim = 'Adam'
args.lr = 5e-5
args.lrs = 'cosine'
args.t_max = 8
args.factor = 0.6
args.patience = 3
args.min_lr = 1e-6
args.iter_val = 200
args.num_epochs = 50
args.batch_size = 512
args.val_batch_size = 512

In [22]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size)

(200840, 5)
(200840, 32332)
(180756, 5) (20084, 5)


In [23]:
model, model_file = create_model(args)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


model file: ./models/se_resnext50_32x4d/best_model.pth, exist: True
loading ./models/se_resnext50_32x4d/best_model.pth...


In [39]:
train(args) # no any augmentation

{'recall': 0.9671440534027664, 'recall_grapheme': 0.9493775668155872, 'recall_vowel': 0.9814981361011735, 'recall_consonant': 0.9883229438787181, 'acc_grapheme': 0.9517028480382393, 'acc_vowel': 0.986705835490938, 'acc_consonant': 0.9859589723162716, 'loss_grapheme': 0.16853864101317984, 'loss_vowel': 0.06034584744378785, 'loss_consonant': 0.04872297896400578}
    0 | 0.000010 | 051200/180756 | 0.0994 | 0.0825 |val: {'recall': 0.9740534755083553, 'recall_grapheme': 0.9603027657701281, 'recall_vowel': 0.9860325981284788, 'recall_consonant': 0.9895757723646864, 'acc_grapheme': 0.9601672973511253, 'acc_vowel': 0.988797052380004, 'acc_consonant': 0.9891953794064927, 'loss_grapheme': 0.13787380800473217, 'loss_vowel': 0.04863242665222265, 'loss_consonant': 0.03757896084539493}
** saved
    0 | 0.000010 | 102400/180756 | 0.0460 | 0.0741 |val: {'recall': 0.9749220658515402, 'recall_grapheme': 0.9619530557647916, 'recall_vowel': 0.986462546838192, 'recall_consonant': 0.9893196050383853, 'acc_g

    5 | 0.000004 | 085248/180756 | 0.0069 | 0.0100 |

KeyboardInterrupt: 

In [48]:
train(args)

{'recall': 0.976747, 'recall_grapheme': 0.965267, 'recall_vowel': 0.986704, 'recall_consonant': 0.989748, 'acc_grapheme': 0.963802, 'acc_vowel': 0.990092, 'acc_consonant': 0.989743, 'loss_grapheme': 0.135432, 'loss_vowel': 0.048011, 'loss_consonant': 0.039993}
    0 | 0.000010 | 051200/180756 | 4.7450 | 4.0992 |
val: {'recall': 0.963693, 'recall_grapheme': 0.947814, 'recall_vowel': 0.974848, 'recall_consonant': 0.984297, 'acc_grapheme': 0.928152, 'acc_vowel': 0.962856, 'acc_consonant': 0.969478, 'loss_grapheme': 0.463083, 'loss_vowel': 0.247168, 'loss_consonant': 0.142234}
    0 | 0.000010 | 102400/180756 | 3.7406 | 3.6959 |
val: {'recall': 0.970157, 'recall_grapheme': 0.954592, 'recall_vowel': 0.982368, 'recall_consonant': 0.989077, 'acc_grapheme': 0.945877, 'acc_vowel': 0.978391, 'acc_consonant': 0.983967, 'loss_grapheme': 0.372751, 'loss_vowel': 0.206895, 'loss_consonant': 0.1229}
    0 | 0.000010 | 153600/180756 | 4.4351 | 3.5165 |
val: {'recall': 0.970651, 'recall_grapheme': 0.955

KeyboardInterrupt: 

In [63]:
train(args) # mixup no augmentation

{'recall': 0.976747, 'recall_grapheme': 0.965267, 'recall_vowel': 0.986704, 'recall_consonant': 0.989748, 'acc_grapheme': 0.963802, 'acc_vowel': 0.990092, 'acc_consonant': 0.989743, 'loss_grapheme': 0.135432, 'loss_vowel': 0.048011, 'loss_consonant': 0.039993}
    0 | 0.000048 | 102400/180756 | 2.6321 | 3.3057 |
val: {'recall': 0.970491, 'recall_grapheme': 0.954772, 'recall_vowel': 0.983103, 'recall_consonant': 0.989318, 'acc_grapheme': 0.952251, 'acc_vowel': 0.98322, 'acc_consonant': 0.985162, 'loss_grapheme': 0.347249, 'loss_vowel': 0.173147, 'loss_consonant': 0.115511}
    1 | 0.000043 | 024064/180756 | 4.2951 | 2.7619 |
val: {'recall': 0.968969, 'recall_grapheme': 0.951771, 'recall_vowel': 0.982496, 'recall_consonant': 0.989839, 'acc_grapheme': 0.950707, 'acc_vowel': 0.983021, 'acc_consonant': 0.984565, 'loss_grapheme': 0.34995, 'loss_vowel': 0.167536, 'loss_consonant': 0.112798}
    1 | 0.000035 | 126464/180756 | 4.1030 | 2.9337 |
val: {'recall': 0.969143, 'recall_grapheme': 0.951

   14 | 0.000008 | 132096/180756 | 1.4441 | 2.1436 |
val: {'recall': 0.975953, 'recall_grapheme': 0.963098, 'recall_vowel': 0.986712, 'recall_consonant': 0.990902, 'acc_grapheme': 0.960715, 'acc_vowel': 0.9882, 'acc_consonant': 0.988996, 'loss_grapheme': 0.253276, 'loss_vowel': 0.126174, 'loss_consonant': 0.089953}
   15 | 0.000016 | 053760/180756 | 1.8645 | 2.1354 |
val: {'recall': 0.975832, 'recall_grapheme': 0.962598, 'recall_vowel': 0.98682, 'recall_consonant': 0.991312, 'acc_grapheme': 0.960964, 'acc_vowel': 0.988747, 'acc_consonant': 0.989046, 'loss_grapheme': 0.227563, 'loss_vowel': 0.114589, 'loss_consonant': 0.087643}
   15 | 0.000025 | 156160/180756 | 2.1907 | 2.1694 |
val: {'recall': 0.975313, 'recall_grapheme': 0.962305, 'recall_vowel': 0.986477, 'recall_consonant': 0.990162, 'acc_grapheme': 0.959669, 'acc_vowel': 0.987901, 'acc_consonant': 0.988498, 'loss_grapheme': 0.262925, 'loss_vowel': 0.130997, 'loss_consonant': 0.0935}
   16 | 0.000035 | 077824/180756 | 2.8184 | 2.12

   29 | 0.000025 | 083456/180756 | 2.1302 | 1.8212 |
val: {'recall': 0.975264, 'recall_grapheme': 0.963009, 'recall_vowel': 0.985824, 'recall_consonant': 0.989215, 'acc_grapheme': 0.962259, 'acc_vowel': 0.988847, 'acc_consonant': 0.989395, 'loss_grapheme': 0.25456, 'loss_vowel': 0.112377, 'loss_consonant': 0.092488}
   30 | 0.000016 | 005120/180756 | 1.9771 | 1.8698 |
val: {'recall': 0.976228, 'recall_grapheme': 0.964582, 'recall_vowel': 0.986567, 'recall_consonant': 0.989181, 'acc_grapheme': 0.963304, 'acc_vowel': 0.988797, 'acc_consonant': 0.990042, 'loss_grapheme': 0.227433, 'loss_vowel': 0.107885, 'loss_consonant': 0.087771}
   30 | 0.000008 | 107520/180756 | 2.0428 | 1.8308 |
val: {'recall': 0.976243, 'recall_grapheme': 0.964728, 'recall_vowel': 0.9871, 'recall_consonant': 0.988418, 'acc_grapheme': 0.964499, 'acc_vowel': 0.988996, 'acc_consonant': 0.989544, 'loss_grapheme': 0.212259, 'loss_vowel': 0.102034, 'loss_consonant': 0.078471}
   31 | 0.000003 | 029184/180756 | 2.1366 | 1.

   44 | 0.000043 | 034816/180756 | 2.4030 | 1.6295 |
val: {'recall': 0.977254, 'recall_grapheme': 0.965811, 'recall_vowel': 0.986481, 'recall_consonant': 0.990911, 'acc_grapheme': 0.965296, 'acc_vowel': 0.989245, 'acc_consonant': 0.990092, 'loss_grapheme': 0.19666, 'loss_vowel': 0.09188, 'loss_consonant': 0.075409}
   44 | 0.000048 | 137216/180756 | 1.4246 | 1.7253 |
val: {'recall': 0.97696, 'recall_grapheme': 0.965599, 'recall_vowel': 0.98644, 'recall_consonant': 0.990203, 'acc_grapheme': 0.9644, 'acc_vowel': 0.988797, 'acc_consonant': 0.990888, 'loss_grapheme': 0.213592, 'loss_vowel': 0.102283, 'loss_consonant': 0.083357}
   45 | 0.000050 | 058880/180756 | 2.3604 | 1.7133 |
val: {'recall': 0.97637, 'recall_grapheme': 0.9649, 'recall_vowel': 0.984253, 'recall_consonant': 0.991427, 'acc_grapheme': 0.962756, 'acc_vowel': 0.9881, 'acc_consonant': 0.990241, 'loss_grapheme': 0.216874, 'loss_vowel': 0.094685, 'loss_consonant': 0.082476}
   45 | 0.000048 | 161280/180756 | 1.5262 | 1.7388 |
v

In [76]:
train(args) # mixup with augmentation 9782

{'recall': 0.978264, 'recall_grapheme': 0.967822, 'recall_vowel': 0.986796, 'recall_consonant': 0.990615, 'acc_grapheme': 0.966789, 'acc_vowel': 0.989693, 'acc_consonant': 0.990689, 'loss_grapheme': 0.181179, 'loss_vowel': 0.087153, 'loss_consonant': 0.066685}
    0 | 0.000048 | 102400/180756 | 1.0828 | 1.7958 |
val: {'recall': 0.975461, 'recall_grapheme': 0.962395, 'recall_vowel': 0.985495, 'recall_consonant': 0.991559, 'acc_grapheme': 0.96201, 'acc_vowel': 0.988946, 'acc_consonant': 0.989295, 'loss_grapheme': 0.226953, 'loss_vowel': 0.114416, 'loss_consonant': 0.077727}
    1 | 0.000043 | 024064/180756 | 0.8055 | 1.9220 |
val: {'recall': 0.974818, 'recall_grapheme': 0.963383, 'recall_vowel': 0.985371, 'recall_consonant': 0.987136, 'acc_grapheme': 0.960466, 'acc_vowel': 0.988498, 'acc_consonant': 0.988946, 'loss_grapheme': 0.260732, 'loss_vowel': 0.106805, 'loss_consonant': 0.088922}
    1 | 0.000035 | 126464/180756 | 2.0229 | 1.7724 |
val: {'recall': 0.974556, 'recall_grapheme': 0.96

   14 | 0.000008 | 132096/180756 | 1.6636 | 1.6448 |
val: {'recall': 0.977799, 'recall_grapheme': 0.966922, 'recall_vowel': 0.985832, 'recall_consonant': 0.991521, 'acc_grapheme': 0.967138, 'acc_vowel': 0.989046, 'acc_consonant': 0.991336, 'loss_grapheme': 0.196955, 'loss_vowel': 0.091504, 'loss_consonant': 0.069337}
   15 | 0.000016 | 053760/180756 | 1.7029 | 1.6748 |
val: {'recall': 0.978034, 'recall_grapheme': 0.966968, 'recall_vowel': 0.986587, 'recall_consonant': 0.991614, 'acc_grapheme': 0.967935, 'acc_vowel': 0.989295, 'acc_consonant': 0.991585, 'loss_grapheme': 0.196397, 'loss_vowel': 0.095467, 'loss_consonant': 0.071954}
   15 | 0.000025 | 156160/180756 | 2.1143 | 1.6859 |
val: {'recall': 0.976275, 'recall_grapheme': 0.964587, 'recall_vowel': 0.984895, 'recall_consonant': 0.99103, 'acc_grapheme': 0.964848, 'acc_vowel': 0.989096, 'acc_consonant': 0.991087, 'loss_grapheme': 0.196546, 'loss_vowel': 0.092777, 'loss_consonant': 0.068522}
   16 | 0.000035 | 077824/180756 | 1.6581 | 

KeyboardInterrupt: 

In [24]:
train(args) # augmentation without mixup 9828

{'recall': 0.978264, 'recall_grapheme': 0.967822, 'recall_vowel': 0.986796, 'recall_consonant': 0.990615, 'acc_grapheme': 0.966789, 'acc_vowel': 0.989693, 'acc_consonant': 0.990689, 'loss_grapheme': 0.181179, 'loss_vowel': 0.087153, 'loss_consonant': 0.066685}


/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    0 | 0.000048 | 102400/180756 | 0.1216 | 0.1066 |
val: {'recall': 0.977223, 'recall_grapheme': 0.966235, 'recall_vowel': 0.987834, 'recall_consonant': 0.988588, 'acc_grapheme': 0.967088, 'acc_vowel': 0.989942, 'acc_consonant': 0.990191, 'loss_grapheme': 0.123551, 'loss_vowel': 0.044626, 'loss_consonant': 0.03469}
    1 | 0.000043 | 024064/180756 | 0.1011 | 0.0690 |
val: {'recall': 0.978634, 'recall_grapheme': 0.96837, 'recall_vowel': 0.986834, 'recall_consonant': 0.990964, 'acc_grapheme': 0.967536, 'acc_vowel': 0.990241, 'acc_consonant': 0.990341, 'loss_grapheme': 0.121514, 'loss_vowel': 0.043641, 'loss_consonant': 0.036495}
** saved
    1 | 0.000035 | 126464/180756 | 0.0582 | 0.0694 |
val: {'recall': 0.980114, 'recall_grapheme': 0.970052, 'recall_vowel': 0.988605, 'recall_consonant': 0.991749, 'acc_grapheme': 0.969777, 'acc_vowel': 0.989793, 'acc_consonant': 0.991436, 'loss_grapheme': 0.114751, 'loss_vowel': 0.044889, 'loss_consonant': 0.032215}
** saved
    2 | 0.000026 | 048128/1

   15 | 0.000016 | 053760/180756 | 0.0345 | 0.0179 |
val: {'recall': 0.981747, 'recall_grapheme': 0.973284, 'recall_vowel': 0.98907, 'recall_consonant': 0.99135, 'acc_grapheme': 0.971769, 'acc_vowel': 0.991336, 'acc_consonant': 0.991735, 'loss_grapheme': 0.120667, 'loss_vowel': 0.046991, 'loss_consonant': 0.037535}
   15 | 0.000025 | 156160/180756 | 0.0267 | 0.0203 |
val: {'recall': 0.980073, 'recall_grapheme': 0.972929, 'recall_vowel': 0.986836, 'recall_consonant': 0.987597, 'acc_grapheme': 0.971022, 'acc_vowel': 0.990291, 'acc_consonant': 0.991187, 'loss_grapheme': 0.125328, 'loss_vowel': 0.051576, 'loss_consonant': 0.042777}
   16 | 0.000035 | 077824/180756 | 0.0336 | 0.0288 |
val: {'recall': 0.977407, 'recall_grapheme': 0.966984, 'recall_vowel': 0.988511, 'recall_consonant': 0.987149, 'acc_grapheme': 0.966989, 'acc_vowel': 0.99044, 'acc_consonant': 0.99049, 'loss_grapheme': 0.136528, 'loss_vowel': 0.050481, 'loss_consonant': 0.039815}
   16 | 0.000043 | 180224/180756 | 0.0312 | 0.0

   30 | 0.000016 | 005120/180756 | 0.0040 | 0.0095 |
val: {'recall': 0.980807, 'recall_grapheme': 0.971388, 'recall_vowel': 0.989321, 'recall_consonant': 0.991133, 'acc_grapheme': 0.970773, 'acc_vowel': 0.991386, 'acc_consonant': 0.991635, 'loss_grapheme': 0.131566, 'loss_vowel': 0.046659, 'loss_consonant': 0.043141}
   30 | 0.000008 | 107520/180756 | 0.0062 | 0.0092 |
val: {'recall': 0.981161, 'recall_grapheme': 0.972228, 'recall_vowel': 0.989257, 'recall_consonant': 0.990932, 'acc_grapheme': 0.970773, 'acc_vowel': 0.991187, 'acc_consonant': 0.991486, 'loss_grapheme': 0.130839, 'loss_vowel': 0.04676, 'loss_consonant': 0.04234}
   31 | 0.000003 | 029184/180756 | 0.0045 | 0.0078 |
val: {'recall': 0.981978, 'recall_grapheme': 0.973454, 'recall_vowel': 0.989662, 'recall_consonant': 0.991341, 'acc_grapheme': 0.971968, 'acc_vowel': 0.991486, 'acc_consonant': 0.991336, 'loss_grapheme': 0.129607, 'loss_vowel': 0.046659, 'loss_consonant': 0.041849}
   31 | 0.000001 | 131584/180756 | 0.0099 | 0

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe047eb3f80>
Traceback (most recent call last):
  File "/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/mnt/chicm/anaconda3/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/mnt/chicm/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/mnt/chicm/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [70]:
#save_model(model, model_file)

In [ ]:
del model

In [73]:
!ls ./models/se_resnext50_32x4d

best_model.pth
